In [1]:
import geopandas as gpd
import xarray as xr
from datetime import datetime as dt
from os import path
import numpy as np
from matplotlib.path import Path
from scipy.interpolate import interp1d
from pyxlma import coords

In [2]:
date_i_want = dt(2022, 6, 2)


In [12]:
tfm = xr.open_dataset(f'/Volumes/LtgSSD/tobac_saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/Track_features_merges_augmented.zarr',
                            engine='zarr', chunks='auto')
polyline = gpd.read_file(f'/Volumes/LtgSSD/analysis/sam_polyline/{date_i_want.strftime("%Y-%m-%d")}.json')

In [13]:
lon_wide = np.arange(-98.3, -91+.005, .01)
lat_wide = np.arange(25.5, 30+.005, .01)
lon_wide, lat_wide = np.meshgrid(lon_wide, lat_wide)
all_seabreezes_wide = np.zeros((lon_wide.shape[0], lon_wide.shape[1], tfm.time.shape[0]))
seabreeze_indices = []
seabreeze_times = []
zero_indices = []
zero_times = []

In [14]:
radar_lat, radar_lon = 29.47, -95.08
tpcs = coords.TangentPlaneCartesianSystem(ctrLat=radar_lat, ctrLon=radar_lon, ctrAlt=0)
geosys = coords.GeographicSystem()
x2d, y2d = np.meshgrid(tfm.x.data, tfm.y.data)
grid_ecef_coords = tpcs.toECEF(x2d.flatten(), y2d.flatten(), np.zeros_like(x2d).flatten())
grid_lon, grid_lat, _ = geosys.fromECEF(*grid_ecef_coords)
grid_lon = grid_lon.reshape(x2d.shape)
grid_lat = grid_lat.reshape(x2d.shape)

tfm = tfm.assign({'lat' : (('x', 'y'), grid_lat), 'lon' : (('x', 'y'), grid_lon)})

all_seabreezes_ds = xr.zeros_like(tfm.segmentation_mask)

In [15]:
for i, time in enumerate(tfm.time.data):
    this_seabreeze = np.zeros_like(lon_wide)
    time_dt = np.array(time).astype('datetime64[s]').astype(dt).item()
    if time_dt in polyline['index'].values.astype(dt):
        seabreeze_indices.append(i)
        seabreeze_times.append(time)
        this_polyline = polyline[polyline['index'] == time_dt]['geometry'].values[0]
        this_polyline_mpl = Path(np.array(this_polyline.exterior.coords))
        this_seabreeze = this_polyline_mpl.contains_points(np.array([lon_wide.flatten(), lat_wide.flatten()]).T).reshape(lon_wide.shape)
        all_seabreezes_wide[:, :, i] = this_seabreeze
        this_seabreeze_ds = this_polyline_mpl.contains_points(np.array([grid_lon.flatten(), grid_lat.flatten()]).T).reshape(grid_lon.shape)
        all_seabreezes_ds[i, :, :] = this_seabreeze_ds
    else:
        zero_indices.append(i)
        zero_times.append(time)
seabreeze_indices = np.array(seabreeze_indices)
seabreeze_times = np.array(seabreeze_times)
zero_indices = np.array(zero_indices)
zero_times = np.array(zero_times)

In [17]:
# Convert seabreeze_times and zero_times to numeric for interpolation
seabreeze_times_num = seabreeze_times.astype('float64')
zero_times_num = zero_times.astype('float64')

In [18]:
def interp_seabreeze_times(all_seabreezes, seabreeze_indices, seabreeze_times_num, zero_indices, zero_times_num):
    all_seabreezes = all_seabreezes.copy()
    # Interpolation: loop through each grid point (lat, lon) and interpolate over time
    for lat_idx in range(all_seabreezes.shape[0]):
        for lon_idx in range(all_seabreezes.shape[1]):
            # Get the seabreeze field values at valid times for this (lat, lon) grid point
            seabreeze_values = all_seabreezes[lat_idx, lon_idx, seabreeze_indices]
            
            # Skip interpolation if all values are zero
            if np.all(seabreeze_values == 0):
                continue
            
            # Create interpolator for the seabreeze field values based on actual times
            interpolator = interp1d(
                seabreeze_times_num,
                seabreeze_values,
                kind='linear',
                bounds_error=False,
                fill_value=0  # You can choose another fill method if appropriate
            )
            
            # Interpolate for missing times (all-zero slices)
            interpolated_values = interpolator(zero_times_num)
            
            # Update all_seabreezes with interpolated values at zero_indices
            all_seabreezes[lat_idx, lon_idx, zero_indices] = interpolated_values
    return all_seabreezes

In [19]:
all_seabreezes_wide.shape

(451, 731, 283)

In [20]:
all_seabreezes_ds = interp_seabreeze_times(all_seabreezes_ds.T.compute(), seabreeze_indices, seabreeze_times_num, zero_indices, zero_times_num).T

In [21]:
tfm['seabreeze'] = all_seabreezes_ds

In [22]:
feature_seabreeze = xr.zeros_like(tfm.feature)
cell_seabreeze = np.zeros_like(np.meshgrid(tfm.cell, tfm.time))
for i, feat_id in enumerate(tfm.feature.data):
    this_feat = tfm.sel(feature=feat_id)
    this_feat_lon = this_feat.feature_lon.compute()
    this_feat_lat = this_feat.feature_lat.compute()
    ll_dist = ((tfm.lon - this_feat_lon)**2 + (tfm.lat - this_feat_lat)**2)**(0.5)
    ll_dist_min = np.unravel_index(np.argmin(ll_dist.data.flatten()), ll_dist.shape)
    this_feat_time_index = this_feat.feature_time_index.data.compute().item()
    closest_seabreeze = tfm.seabreeze.isel(time=this_feat_time_index)[ll_dist_min]
    feature_seabreeze[i] = closest_seabreeze

In [28]:
for cell_id in tfm.feature_parent_cell_id.data.compute():
    if cell_id == -1:
        continue
    print(cell_id)

4
6
8
10
12
16
19
20
21
23
28
30
32
4
6
8
12
10
16
19
20
21
23
30
28
32
4
6
8
12
10
55
19
16
20
21
23
63
30
32
4
6
8
12
10
55
19
16
20
78
21
23
30
28
32
87
6
12
8
10
55
19
16
20
78
96
21
63
30
28
32
87
4
105
6
108
10
55
12
8
19
16
20
78
96
21
30
32
28
87
4
105
6
55
108
12
10
129
130
8
16
96
20
19
21
23
138
30
32
28
87
105
6
144
108
10
129
130
8
16
96
20
12
19
21
23
138
63
30
28
32
156
87
105
6
144
108
130
10
8
96
16
20
19
164
21
138
63
30
32
156
28
87
105
178
78
108
10
181
182
16
20
19
183
21
23
138
63
30
156
32
28
105
178
78
129
108
10
181
182
16
20
19
183
21
23
138
63
30
196
156
28
144
105
130
108
181
129
10
182
16
164
78
20
183
208
21
23
138
63
215
30
217
218
156
28
144
105
130
181
108
223
224
10
6
182
16
183
208
232
21
23
235
236
215
30
196
217
218
156
238
28
178
105
242
223
130
181
224
243
10
129
182
16
245
183
232
208
21
23
235
236
156
30
217
32
238
28
105
242
130
223
224
256
16
245
182
183
208
232
23
21
235
236
156
196
217
30
32
28
238
144
105
130
223
224
10
243
256
245
164
182


In [29]:
tfm.feature_parent_cell_id.data.compute()

array([  -1,   -1,   -1, ..., 5110,   -1,   -1])

In [36]:
interp_wide

NameError: name 'interp_wide' is not defined